# New clusters annotation

## Libraries imports

In [ ]:
import pandas as pd
import numpy as np

from Bio import SeqIO
from Bio.Seq import Seq

## Data

In [ ]:
DATA_FOLDER = "../Data/Sequences/Clusters reference sequences/"
OUTPUT_FOLDER = "../Data/New clusters annotations/"

K12_FILE = DATA_FOLDER+"K12.faa" # K12 sequences (fasta file)

## Creating queries for vsearch searches

In [ ]:
record = SeqIO.to_dict(SeqIO.parse(K12_FILE, "fasta"))
for seq in record.values():
    SeqIO.write([seq],OUTPUT_FOLDER+(seq.id).split(".1_")[1]+".fasta","fasta")